In [ ]:
import pandas as pd
import numpy as np

titles = ['ADT' ,  'AHM' ,  'AoR' ,  'OE1O' , 'Anæstesidata' , 'Anæstesihændelse' ,  'Blødning' ,  'ca_vitale_værdier' ,   'Dialyse' ,  'EKG_data' ,  'Ekkokardiografi' ,  'Højde' ,  'intellispace' ,  'ITA' ,  'KAG_data' ,  'medicin' ,  'Pleuradræn' ,  'Populationen' ,  'Problemliste' , 'Respirator_data' ,  'Samlet_udskillelse' ,  'Spiromitri' ,  'Total_indgift' ,  'Urin' ,  'Vægt' ,  'viewpoint' ]

for title in titles:
    parquet_file = f'Processed_parquet_real\{title}.parquet.gzip'
    df = pd.read_parquet(parquet_file)
    globals()[title] = df

Spiromitri.drop(index=[2685,2686,2687,2688],inplace=True) # removes the rows with / in the values
Lab_svar = pd.read_csv(r'opdateret_data\Lab_svar.csv')
Lab_svar_1=Lab_svar[:2830372]
Lab_svar_2 = Lab_svar[2830372:]

Lab_svar_1 = Lab_svar_1.rename(columns={ 'V2' : 'What is being measured' ,  'V3' :  'Timestamp' ,  'V4' :  'Results' })
Lab_svar_1 = Lab_svar_1.dropna(subset=['Results'])
Lab_svar_1['Results'] = pd.to_numeric(Lab_svar_1['Results'], errors='coerce')
Lab_svar_1 = Lab_svar_1.groupby(['Timestamp','IDno','What is being measured']).Results.mean().reset_index()
Lab_svar_1 = Lab_svar_1.pivot(index = ['IDno','Timestamp'], columns=['What is being measured'],values='Results')
Lab_svar_1.reset_index(inplace=True)
Lab_svar_1['Timestamp'] = pd.to_datetime(Lab_svar_1['Timestamp'])

Lab_svar_2 = Lab_svar_2.rename(columns={ 'V2' : 'What is being measured' ,  'V3' :  'Timestamp' ,  'V4' :  'Results' })
Lab_svar_2 = Lab_svar_2.dropna(subset=['Results'])
Lab_svar_2['Results'] = pd.to_numeric(Lab_svar_2['Results'], errors='coerce')
Lab_svar_2 = Lab_svar_2.groupby(['Timestamp','IDno','What is being measured']).Results.mean().reset_index()
Lab_svar_2 = Lab_svar_2.pivot(index = ['IDno','Timestamp'], columns=['What is being measured'],values='Results')
Lab_svar_2.reset_index(inplace=True)
Lab_svar_2['Timestamp'] = pd.to_datetime(Lab_svar_2['Timestamp'])

Spiromitri=Spiromitri.rename(columns={"Resultatdato": "Timestamp"})

pre_liste = []

In [ ]:
# fjerner alle rækkerne hvor Induktion står lige efter Stop data indsamling da de derfor ikke indgår i åben hjertekirugi
test_hændelse = Anæstesihændelse.copy()

rows_to_remove = []
for i in range(len(test_hændelse) - 1):
    if test_hændelse['Hændelse'].iloc[i] == 'Induktion' and test_hændelse['Hændelse'].iloc[i + 1] == 'Stop Data Indsamling':
        rows_to_remove.extend([i, i + 1])

filtered_test_hændelse = test_hændelse.drop(rows_to_remove).reset_index(drop=True)

In [ ]:
# Assuming Populationen and filtered_test_hændelse are DataFrames
IDs_listx = Populationen['IDno'].unique()

no_induktion = []
no_stop = []
nothing = []
more_induktion = []
more_stop = []
missing_events = []
both = []
for ID in IDs_listx:
    # Filter the rows where ID matches and Hændelse is 'Induktion' or 'Stop Data Indsamling'
    event_rows_induk = filtered_test_hændelse[(filtered_test_hændelse['IDno'] == ID) & (filtered_test_hændelse['Hændelse'] == 'Induktion')]
    event_rows_stop = filtered_test_hændelse[(filtered_test_hændelse['IDno'] == ID) & (filtered_test_hændelse['Hændelse'] == 'Stop Data Indsamling')]
    event_rows_bypass_start = filtered_test_hændelse[(filtered_test_hændelse['IDno'] == ID) & (filtered_test_hændelse['Hændelse'] == 'CV Bypass Start')]
    event_rows_aorta_tang_på = filtered_test_hændelse[(filtered_test_hændelse['IDno'] == ID) & (filtered_test_hændelse['Hændelse'] == 'Aorta tang på')]
    event_rows_aorta_tang_af = filtered_test_hændelse[(filtered_test_hændelse['IDno'] == ID) & (filtered_test_hændelse['Hændelse'] == 'Aorta tang af')]
    event_rows_bypass_slut = filtered_test_hændelse[(filtered_test_hændelse['IDno'] == ID) & (filtered_test_hændelse['Hændelse'] == 'CV Bypass slut')]

    # Check if any event is missing
    if (event_rows_bypass_start.empty or event_rows_aorta_tang_på.empty or event_rows_aorta_tang_af.empty or event_rows_bypass_slut.empty):
        missing_events.append(ID)

    if len(event_rows_induk) > 1 and len(event_rows_stop) > 1:
        both.append(ID)

    # Check for more than one row
    if  len(event_rows_induk) > 1:
        if ID not in both:
            more_induktion.append(ID)
    if len(event_rows_stop) > 1:
        if ID not in both:
            more_stop.append(ID)

    # If it has nothing
    if event_rows_induk.empty and event_rows_stop.empty:
        nothing.append(ID)
    
    # Add ID if it doesn't have either 'Induktion' or 'Stop Data Indsamling'
    if event_rows_induk.empty:
        if ID not in nothing:
            no_induktion.append(ID)
    if event_rows_stop.empty:
        if ID not in nothing:
            no_stop.append(ID)

In [ ]:
dataframes = {
    'ADT': ADT,
    'AHM': AHM,
    'AoR': AoR,
    'OE1O': OE1O,
    'Anæstesidata': Anæstesidata,
    'Anæstesihændelse': filtered_test_hændelse,
    'Blødning': Blødning,
    'ca_vitale_værdier': ca_vitale_værdier,
    'Dialyse': Dialyse,
    'EKG_data': EKG_data,
    'Ekkokardiografi': Ekkokardiografi,
    'Højde': Højde,
    'intellispace': intellispace,
    'ITA': ITA,
    'KAG_data': KAG_data,
    'Lab_svar': Lab_svar,
    'medicin': medicin,
    'Pleuradræn': Pleuradræn,
    'Populationen': Populationen,
    'Problemliste': Problemliste,
    'Respirator_data': Respirator_data,
    'Samlet_udskillelse': Samlet_udskillelse,
    'Spiromitri': Spiromitri,
    'Total_indgift': Total_indgift,
    'Urin': Urin,
    'Vægt': Vægt,
    'viewpoint': viewpoint
}

def check_timestamp_header_and_type(df_dict):
    results = []
    for name, df in df_dict.items():
        if 'Timestamp' in df.columns:
            timestamp_type = df['Timestamp'].dtype
            if timestamp_type == np.dtype('O'):
                try:
                    # Try parsing with the default format
                    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
                except Exception as e_default:
                    print(f"Default parsing failed for DataFrame '{name}' with error: {e_default}")
                    try:
                        # Try parsing with dayfirst=True
                        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True, dayfirst=True)
                    except Exception as e_dayfirst:
                        print(f"Parsing with dayfirst=True failed for DataFrame '{name}' with error: {e_dayfirst}")
                        try:
                            # If parsing fails, you can specify a custom format or handle mixed formats
                            # Example: df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d-%m-%Y', utc=True)
                            df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True, errors='coerce')
                            if df['Timestamp'].isnull().any():
                                raise ValueError("Some dates could not be parsed and resulted in NaT values.")
                        except Exception as e_custom:
                            print(f"Custom parsing failed for DataFrame '{name}' with error: {e_custom}")
                            results.append((name, False, None))
                            continue
                timestamp_type = df['Timestamp'].dtype  # Update type after conversion
            results.append((name, True, timestamp_type))
        else:
            results.append((name, False, None))
    return results

# Check each DataFrame in the dictionary for the 'Timestamp' header and its type
results = check_timestamp_header_and_type(dataframes)

In [ ]:
IDs_listx = np.unique(Populationen['IDno'])
IDs_listx = [ID for ID in IDs_listx if ID not in no_induktion and ID not in no_stop and ID not in nothing and ID not in more_induktion and ID not in more_stop and ID not in missing_events and ID not in both]
IDS_dfx = pd.DataFrame({'IDno': IDs_listx})

In [ ]:
Anæstesihændelse_clean = filtered_test_hændelse
for id in IDs_listx:
    Anæstesihændelse_id = filtered_test_hændelse[filtered_test_hændelse['IDno'] == id]
    
    # Initialize dictionaries to store indexes of rows to keep
    first_instances = {}
    last_instances = {}
    
    # Initialize a list to store indexes of rows to remove
    rows_to_remove = []

    # Loop through the dataframe to decide which rows to keep or remove
    for index, row in Anæstesihændelse_id.iterrows():
        
        event = row['Hændelse']
        
        # Mark the first instances
        if event == 'Induktion' and 'Induktion' not in first_instances:
            first_instances['Induktion'] = index
        if event == 'CV Bypass Start' and 'CV Bypass Start' not in first_instances:
            first_instances['CV Bypass Start'] = index
        if event == 'Aorta tang på' and 'Aorta tang på' not in first_instances:
            first_instances['Aorta tang på'] = index
        
        # Continuously update the last instances
        if event == 'Aorta tang af':
            last_instances['Aorta tang af'] = index
        if event == 'CV Bypass slut':
            last_instances['CV Bypass slut'] = index
        if event == 'Stop Data Indsamling':
            last_instances['Stop Data Indsamling'] = index
    
    # Create a set of indexes to keep
    indexes_to_keep = set(first_instances.values()).union(last_instances.values())
    
    # Mark rows to remove
    for index in Anæstesihændelse_id.index:
        if index not in indexes_to_keep:
            rows_to_remove.append(index)


    Anæstesihændelse_clean.drop(rows_to_remove, inplace=True)

Load filer. OBS! Dialyse fase 1-4 gemmer ikke, fordi deres dataframes er tomme. Processed versioner er gemt, så det er den, der skal loades i stedet for. Hvis processed versioner er lavet, så load dem i stedet for

In [ ]:
import pandas as pd
import numpy as np
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

# Define the folder path
folder_path = "Phases"

# Define the base file names
base_file_names = [
    # "PhasesAHM_Phase_",
    # "PhasesAnæstesidata_Phase_",
    # "PhasesBlødning_Phase_",
    # "PhasesLab_svar_1_Phase_",
    # "PhasesLab_svar_2_Phase_",
    "PhasesTotal_indgift_Phase_"
]

# "Dialyse_Phase_",

def LoadData(folder_path,base_file_names):
    # Initialize a dictionary to store DataFrames for all phases
    dataframes = {phase: {} for phase in range(1, 6)}

    # Loop through each phase and each base file name to load the CSV files
    for phase in range(1, 6):
        for base_file_name in base_file_names:
            file_path = f"{folder_path}/{base_file_name}{phase}.csv"
            key = base_file_name.split("_Phase_")[0] + f"_Phase_{phase}"
            dataframes[phase][key] = pd.read_csv(file_path)
    return dataframes


In [ ]:
df = LoadData(folder_path=folder_path,base_file_names=base_file_names)

In [ ]:
Total_indgift_Phase_1 = df[1]['PhasesTotal_indgift_Phase_1']
Total_indgift_Phase_2 = df[2]['PhasesTotal_indgift_Phase_2']
Total_indgift_Phase_3 = df[3]['PhasesTotal_indgift_Phase_3']
Total_indgift_Phase_4 = df[4]['PhasesTotal_indgift_Phase_4']
Total_indgift_Phase_5 = df[5]['PhasesTotal_indgift_Phase_5']

In [ ]:
Blødning_Phase_1 = df[1]['PhasesBlødning_Phase_1']
Blødning_Phase_2 = df[2]['PhasesBlødning_Phase_2']
Blødning_Phase_3 = df[3]['PhasesBlødning_Phase_3']
Blødning_Phase_4 = df[4]['PhasesBlødning_Phase_4']
Blødning_Phase_5 = df[5]['PhasesBlødning_Phase_5']

In [ ]:
Anæstesidata_Phase_1 = df[1]['PhasesAnæstesidata_Phase_1']
Anæstesidata_Phase_2 = df[2]['PhasesAnæstesidata_Phase_2']
Anæstesidata_Phase_3 = df[3]['PhasesAnæstesidata_Phase_3']
Anæstesidata_Phase_4 = df[4]['PhasesAnæstesidata_Phase_4']
Anæstesidata_Phase_5 = df[5]['PhasesAnæstesidata_Phase_5']

In [ ]:
Lab_svar_1_Phase_1 = df[1]['PhasesLab_svar_1_Phase_1']
Lab_svar_1_Phase_2 = df[2]['PhasesLab_svar_1_Phase_2']
Lab_svar_1_Phase_3 = df[3]['PhasesLab_svar_1_Phase_3']
Lab_svar_1_Phase_4 = df[4]['PhasesLab_svar_1_Phase_4']
Lab_svar_1_Phase_5 = df[5]['PhasesLab_svar_1_Phase_5']

In [ ]:
Lab_svar_2_Phase_1 = df[1]['PhasesLab_svar_2_Phase_1']
Lab_svar_2_Phase_2 = df[2]['PhasesLab_svar_2_Phase_2']
Lab_svar_2_Phase_3 = df[3]['PhasesLab_svar_2_Phase_3']
Lab_svar_2_Phase_4 = df[4]['PhasesLab_svar_2_Phase_4']
Lab_svar_2_Phase_5 = df[5]['PhasesLab_svar_2_Phase_5']

In [ ]:
AHM_Phase_1 = df[1]['PhasesAHM_Phase_1']
AHM_Phase_2 = df[2]['PhasesAHM_Phase_2']
AHM_Phase_3 = df[3]['PhasesAHM_Phase_3']
AHM_Phase_4 = df[4]['PhasesAHM_Phase_4']
AHM_Phase_5 = df[5]['PhasesAHM_Phase_5']

In [ ]:
folder_path = 'Phases'
base_file_names = ['Processed_AHM_Phase_']
df_AHM = LoadData(folder_path,base_file_names)

Processed_AHM_Phase_1 = df_AHM[1]['Processed_AHM_Phase_1']
Processed_AHM_Phase_2 = df_AHM[2]['Processed_AHM_Phase_2']
Processed_AHM_Phase_3 = df_AHM[3]['Processed_AHM_Phase_3']
Processed_AHM_Phase_4 = df_AHM[4]['Processed_AHM_Phase_4']
Processed_AHM_Phase_5 = df_AHM[5]['Processed_AHM_Phase_5']

In [ ]:
folder_path = 'Phases'
base_file_names = ['Processed_Dialyse_Phase_']
df_Dialyse = LoadData(folder_path,base_file_names)

In [ ]:
Processed_Dialyse_Phase_1 = df_Dialyse[1]['Processed_Dialyse_Phase_1']
Processed_Dialyse_Phase_2 = df_Dialyse[2]['Processed_Dialyse_Phase_2']
Processed_Dialyse_Phase_3 = df_Dialyse[3]['Processed_Dialyse_Phase_3']
Processed_Dialyse_Phase_4 = df_Dialyse[4]['Processed_Dialyse_Phase_4']
Processed_Dialyse_Phase_5 = df_Dialyse[5]['Processed_Dialyse_Phase_5']

In [ ]:
len(Anæstesihændelse_clean['IDno'].unique())

In [ ]:
# funky_ids = []
# for id in IDs_listx:
#     eeeeet = Anæstesihændelse_clean[Anæstesihændelse_clean['IDno'] == id]
#     counts = {}
#     for value in eeeeet['Hændelse']:
#         counts[value] = counts.get(value, 0) + 1
#     keys = list(counts.keys())
#     for key in keys:
#         if counts[key] != 1:
#             #print(f'{key} occurs {counts[key]} times for {id}')
#             funky_ids.append(id)

# funky_ids=list(np.unique(funky_ids))
# IDs_listx = [ID for ID in IDs_listx if ID not in funky_ids]

In [ ]:
def Phases(IDno,Dataframe): # Har indtil videre brugt AHM_processed i stedet for Dataframe
    Hændelsestidspunkt = Anæstesihændelse_clean[Anæstesihændelse_clean['IDno'] == IDno]

    Induktion = Hændelsestidspunkt[Hændelsestidspunkt['Hændelse'] == 'Induktion']['Timestamp'].reset_index(drop = True)[0]
    Bypass_start = (Hændelsestidspunkt[Hændelsestidspunkt['Hændelse'] == 'CV Bypass Start']['Timestamp']).reset_index(drop = True)[0]
    Aorta_tang_på = Hændelsestidspunkt[Hændelsestidspunkt['Hændelse'] == 'Aorta tang på']['Timestamp'].reset_index(drop = True)[0]
    Aorta_tang_af = Hændelsestidspunkt[Hændelsestidspunkt['Hændelse'] == 'Aorta tang af']['Timestamp'].reset_index(drop = True)[0]
    Bypass_slut = Hændelsestidspunkt[Hændelsestidspunkt['Hændelse'] == 'CV Bypass slut']['Timestamp'].reset_index(drop = True)[0]
    Stop_indsaml = Hændelsestidspunkt[Hændelsestidspunkt['Hændelse'] == 'Stop Data Indsamling']['Timestamp'].reset_index(drop = True)[0]
    AHM_ID = Dataframe[Dataframe['IDno'] == IDno]
    AHM_ID = AHM_ID.set_index(['Timestamp'])

    Phase_1 = AHM_ID.loc[Induktion:(Bypass_start)]
    Phase_1 = Phase_1.drop(Phase_1.index[-1])

    Phase_2 = AHM_ID.loc[Bypass_start:(Aorta_tang_på)]
    Phase_2 = Phase_2.drop(Phase_2.index[-1])

    Phase_3 = AHM_ID.loc[Aorta_tang_på:(Aorta_tang_af)]
    Phase_3 = Phase_3.drop(Phase_3.index[-1])

    Phase_4 = AHM_ID.loc[Aorta_tang_af:(Bypass_slut)]
    Phase_4 = Phase_4.drop(Phase_4.index[-1])

    Phase_5 = AHM_ID.loc[Bypass_slut:(Stop_indsaml)]
    return Phase_1, Phase_2,Phase_3,Phase_4,Phase_5

def Aggregate_Phases(id_list, Dataframe):
    phase_1_list = []
    phase_2_list = []
    phase_3_list = []
    phase_4_list = []
    phase_5_list = []

    for IDno in id_list:
        print(IDno)
        Phase_1, Phase_2, Phase_3, Phase_4, Phase_5 = Phases(IDno, Dataframe)
        phase_1_list.append(Phase_1)
        phase_2_list.append(Phase_2)
        phase_3_list.append(Phase_3)
        phase_4_list.append(Phase_4)
        phase_5_list.append(Phase_5)

    phase_1_df = pd.concat(phase_1_list)
    phase_2_df = pd.concat(phase_2_list)
    phase_3_df = pd.concat(phase_3_list)
    phase_4_df = pd.concat(phase_4_list)
    phase_5_df = pd.concat(phase_5_list)

    return phase_1_df, phase_2_df, phase_3_df, phase_4_df, phase_5_df

In [ ]:
def filter_dataframe_by_ids_and_timestamp(IDs_listx, df):
    phase_dfs = {
        'Phase_1': [],
        'Phase_2': [],
        'Phase_3': [],
        'Phase_4': [],
        'Phase_5': []
    }

    for idno in IDs_listx:
        #print(idno)
        # Filter Anæstesihændelse based on ID number
        hændelse_data = Anæstesihændelse_clean[Anæstesihændelse_clean['IDno'] == idno]

        if not hændelse_data.empty:
            # Extract relevant timestamps
            Induktion_time = hændelse_data[hændelse_data['Hændelse'] == 'Induktion']['Timestamp'].values
            Bypass_start_time = hændelse_data[hændelse_data['Hændelse'] == 'CV Bypass Start']['Timestamp'].values
            Aorta_tang_på_time = hændelse_data[hændelse_data['Hændelse'] == 'Aorta tang på']['Timestamp'].values
            Aorta_tang_af_time = hændelse_data[hændelse_data['Hændelse'] == 'Aorta tang af']['Timestamp'].values
            Bypass_slut_time = hændelse_data[hændelse_data['Hændelse'] == 'CV Bypass slut']['Timestamp'].values
            Stop_indsaml_time = hændelse_data[hændelse_data['Hændelse'] == 'Stop Data Indsamling']['Timestamp'].values

            if len(Induktion_time) > 0 and len(Bypass_start_time) > 0 and len(Aorta_tang_på_time) > 0 and len(Aorta_tang_af_time) > 0 and len(Bypass_slut_time) > 0 and len(Stop_indsaml_time) > 0:
                current_df = df[df['IDno'] == idno]
                Induktion_time_utc = pd.Timestamp(Induktion_time[0], tz='UTC')
                Bypass_start_time_utc = pd.Timestamp(Bypass_start_time[0], tz='UTC')
                Aorta_tang_på_time_utc = pd.Timestamp(Aorta_tang_på_time[0], tz='UTC')
                Aorta_tang_af_time_utc = pd.Timestamp(Aorta_tang_af_time[0], tz='UTC')
                Bypass_slut_time_utc = pd.Timestamp(Bypass_slut_time[0], tz='UTC')
                Stop_indsaml_time_utc = pd.Timestamp(Stop_indsaml_time[0], tz='UTC')

                # Phase 2: Filter between Induktion and Bypass_start
                phase_1 = current_df[(current_df['Timestamp'] >= Induktion_time_utc) & (current_df['Timestamp'] < Bypass_start_time_utc)]
                phase_dfs['Phase_1'].append(phase_1)

                # Phase 3: Filter between Bypass_start and Aorta_tang_på
                phase_2 = current_df[(current_df['Timestamp'] >= Bypass_start_time_utc) & (current_df['Timestamp'] < Aorta_tang_på_time_utc)]
                phase_dfs['Phase_2'].append(phase_2)

                # Phase 4: Filter between Aorta_tang_på and Aorta_tang_af
                phase_3 = current_df[(current_df['Timestamp'] >= Aorta_tang_på_time_utc) & (current_df['Timestamp'] < Aorta_tang_af_time_utc)]
                phase_dfs['Phase_3'].append(phase_3)

                # Phase 5: Filter between Aorta_tang_af and Bypass_slut
                phase_4 = current_df[(current_df['Timestamp'] >= Aorta_tang_af_time_utc) & (current_df['Timestamp'] < Bypass_slut_time_utc)]
                phase_dfs['Phase_4'].append(phase_4)

                # Phase 6: Filter between Bypass_slut and Stop_indsaml
                phase_5 = current_df[(current_df['Timestamp'] >= Bypass_slut_time_utc) & (current_df['Timestamp'] <= Stop_indsaml_time_utc)]
                phase_dfs['Phase_5'].append(phase_5)

    # Concatenate all phases
    for phase_name in phase_dfs:
        phase_dfs[phase_name] = pd.concat(phase_dfs[phase_name]) if phase_dfs[phase_name] else pd.DataFrame()

    return phase_dfs


Fejlkilde hvis data indsamling først er blevet stoppet offficielt lang tid efter operationen kan det påvirke vores data. Kan tage gennemsnit og så afslutte operationer konsekvent et par timer efter CV bypass slut, dette kan nok også gøres for dem der mangler en stop data indsamling

In [ ]:
# List of DataFrames
Dataframes_intra = [AHM, Anæstesidata, Blødning, Dialyse,Lab_svar_1, Lab_svar_2, Total_indgift]

# List of DataFrame names
df_names = ['AHM', 'Anæstesidata', 'Blødning', 'Dialyse','Lab_svar_1', 'Lab_svar_2', 'Total_indgift']

# Unique IDs from Populationen and filter them

# Initialize a dictionary to store filtered DataFrames
filtered_dataframes = {}

# Process each DataFrame in Dataframes_pre
for df_name, df in zip(df_names, Dataframes_intra):
    # Ensure the DataFrame has the 'IDno' column
    print(df_name)
    if 'IDno' in df.columns:
        # Apply the filtering function to each DataFrame
        filtered_df = filter_dataframe_by_ids_and_timestamp(IDs_listx, df)
        filtered_dataframes[df_name] = filtered_df
    else:
        print(f"DataFrame {df_name} does not contain 'IDno' column.")

# Validate if the length of unique IDs in filtered_Population matches IDs_listx
if 'Populationen' in filtered_dataframes:
    unique_ids_in_filtered_population = filtered_dataframes['Populationen']['IDno'].nunique()
    if unique_ids_in_filtered_population != len(IDs_listx):
        print(f"Mismatch: {unique_ids_in_filtered_population} unique IDs in filtered_Population, expected {len(IDs_listx)}.")

# Extract the filtered DataFrames back to individual variables if needed
filtered_AHM =filtered_dataframes.get('AHM',None)
filtered_Anæstesidata =filtered_dataframes.get('Anæstesidata',None)
# filtered_Anæstesihændelse =filtered_dataframes.get('Anæstesihændelse',None)
filtered_Blødning =filtered_dataframes.get('Blødning',None)
filtered_Dialyse =filtered_dataframes.get('Dialyse',None)
filtered_Lab_svar_1 =filtered_dataframes.get('Lab_svar_1',None)
filtered_Lab_svar_2 =filtered_dataframes.get('Lab_svar_2',None)
filtered_Total_indgift =filtered_dataframes.get('Total_indgift',None)

In [ ]:
AHM_Phase_1 = filtered_AHM['Phase_1']
Anæstesidata_Phase_1 = filtered_Anæstesidata['Phase_1']
Blødning_Phase_1 = filtered_Blødning['Phase_1']
Dialyse_Phase_1 = filtered_Dialyse['Phase_1']
Lab_svar_1_Phase_1 = filtered_Lab_svar_1['Phase_1']
Lab_svar_2_Phase_1 = filtered_Lab_svar_2['Phase_1']
Total_indgift_Phase_1 = filtered_Total_indgift['Phase_1']

In [ ]:
AHM_Phase_2 = filtered_AHM['Phase_2']
Anæstesidata_Phase_2 = filtered_Anæstesidata['Phase_2']
Blødning_Phase_2 = filtered_Blødning['Phase_2']
Dialyse_Phase_2 = filtered_Dialyse['Phase_2']
Lab_svar_1_Phase_2 = filtered_Lab_svar_1['Phase_2']
Lab_svar_2_Phase_2 = filtered_Lab_svar_2['Phase_2']
Total_indgift_Phase_2 = filtered_Total_indgift['Phase_2']

In [ ]:
AHM_Phase_3 = filtered_AHM['Phase_3']
Anæstesidata_Phase_3 = filtered_Anæstesidata['Phase_3']
Blødning_Phase_3 = filtered_Blødning['Phase_3']
Dialyse_Phase_3 = filtered_Dialyse['Phase_3']
Lab_svar_1_Phase_3 = filtered_Lab_svar_1['Phase_3']
Lab_svar_2_Phase_3 = filtered_Lab_svar_2['Phase_3']
Total_indgift_Phase_3 = filtered_Total_indgift['Phase_3']

In [ ]:
AHM_Phase_4 = filtered_AHM['Phase_4']
Anæstesidata_Phase_4 = filtered_Anæstesidata['Phase_4']
Blødning_Phase_4 = filtered_Blødning['Phase_4']
Dialyse_Phase_4 = filtered_Dialyse['Phase_4']
Lab_svar_1_Phase_4 = filtered_Lab_svar_1['Phase_4']
Lab_svar_2_Phase_4 = filtered_Lab_svar_2['Phase_4']
Total_indgift_Phase_4 = filtered_Total_indgift['Phase_4']

In [ ]:
AHM_Phase_5 = filtered_AHM['Phase_5']
Anæstesidata_Phase_5 = filtered_Anæstesidata['Phase_5']
Blødning_Phase_5 = filtered_Blødning['Phase_5']
Dialyse_Phase_5 = filtered_Dialyse['Phase_5']
Lab_svar_1_Phase_5 = filtered_Lab_svar_1['Phase_5']
Lab_svar_2_Phase_5 = filtered_Lab_svar_2['Phase_5']
Total_indgift_Phase_5 = filtered_Total_indgift['Phase_5']

In [ ]:
# Define paths where you want to save the CSV files
output_path_phases = 'Phases'

# Function to save phases
def save_phases(filtered_data, dataset_name):
    for key, phase_df in filtered_data.items():
        if not phase_df.empty:
            phase_df.to_csv(f"{output_path_phases}/{dataset_name}_{key}.csv", index=False)



In [ ]:
len(IDS_dfx)

In [ ]:
# Save phases for Anæstesidata
save_phases(filtered_Anæstesidata, 'Anæstesidata')

# Save phases for AHM
save_phases(filtered_AHM, 'AHM')

# Save phases for Blødning
save_phases(filtered_Blødning, 'Blødning')

# Save phases for Dialyse
save_phases(filtered_Dialyse, 'Dialyse')

# Save phases for Lab_svar_1
save_phases(filtered_Lab_svar_1, 'Lab_svar_1')

# Save phases for Lab_svar_2
save_phases(filtered_Lab_svar_2, 'Lab_svar_2')

# Save phases for Total_indgift
save_phases(filtered_Total_indgift, 'Total_indgift')


Dialyse

In [ ]:
def DialyseProcessing(filtered_Dialyse):
    Processed_Dialyse = filtered_Dialyse.drop(columns =['Blodflow - indstille','Mål for væsketræk', 'Timestamp']) #Vi sletter mål for væsketræk, da det kun er målt 61 gange
    encoded_dia1 = pd.get_dummies(filtered_Dialyse['Filtersæt'])
    encoded_dia2 = pd.get_dummies(filtered_Dialyse['Væsketype dialysat'])
    encoded_dia3 = pd.get_dummies(filtered_Dialyse['CRRT modus'])
    df = pd.concat([Processed_Dialyse, encoded_dia1.astype(int), encoded_dia2.astype(int), encoded_dia3.astype(int)], axis=1)
    df = df.drop(columns=['Filtersæt','Væsketype dialysat', 'CRRT modus'])
    Processed_Dialyse = df
    Processed_Dialyse['Valg af antikoagulant til Multifiltrate: CVVHD eller CVVHDF'] = Processed_Dialyse['Valg af antikoagulant til Multifiltrate: CVVHD eller CVVHDF'].apply(lambda x: 1 if pd.notna(x) else 0)
    # List of columns to calculate statistics for
    columns_of_interest = [
        'Dialysatvæske hastighed - indstillet',
        'Væskefjernelse fra patienten - indstillet',
        'Væskefjernelse fra patienten - aflæst',
        'Væsketræk siden sidste aflæsning (I/U)'
    ]

    # Calculate mean and variance for each "IDno" and column
    mean_df = Processed_Dialyse.groupby('IDno')[columns_of_interest].mean().add_suffix('_mean')
    var_df = Processed_Dialyse.groupby('IDno')[columns_of_interest].var().add_suffix('_var')

    # Merge the mean and variance dataframes with the original dataframe
    Processed_Dialyse = Processed_Dialyse.merge(mean_df, on='IDno', how='left')
    Processed_Dialyse = Processed_Dialyse.merge(var_df, on='IDno', how='left')
    Processed_Dialyse = Processed_Dialyse.drop(columns = columns_of_interest)
    Processed_Dialyse = Processed_Dialyse.groupby('IDno').max().reset_index()

    # Identify IDno values that are in Populationen but not in Processed_Dialyse
    missing_idnos = IDS_dfx[~IDS_dfx['IDno'].isin(Processed_Dialyse['IDno'])]['IDno']

    # Create a dataframe with these missing IDnos and 0 in every other input
    missing_data = pd.DataFrame(missing_idnos, columns=['IDno'])
    for col in Processed_Dialyse.columns:
        if col != 'IDno':
            missing_data[col] = 0

    # Append this missing data to Processed_Dialyse
    Processed_Dialyse = pd.concat([Processed_Dialyse, missing_data], ignore_index=True)

    # Replace NaN values with 0
    Processed_Dialyse.fillna(0, inplace=True)
    Processed_Dialyse = Processed_Dialyse.groupby('IDno').max().reset_index()
    pre_liste.append(Processed_Dialyse)
    return Processed_Dialyse

In [ ]:
Processed_Dialyse_Phase_1 = DialyseProcessing(Dialyse_Phase_1)
Processed_Dialyse_Phase_2 = DialyseProcessing(Dialyse_Phase_2)
Processed_Dialyse_Phase_3 = DialyseProcessing(Dialyse_Phase_3)
Processed_Dialyse_Phase_4 = DialyseProcessing(Dialyse_Phase_4)
Processed_Dialyse_Phase_5 = DialyseProcessing(Dialyse_Phase_5)

In [ ]:
def save_single_phase(output_path_phases,dataset_name,phase_df):
    phase_df.to_csv(f"{output_path_phases}/{dataset_name}_{key}.csv", index=False)

In [ ]:
Processed_Dialyse_Phase_1.to_csv("Processed_Dialyse_Phase_1.csv", index=False)
Processed_Dialyse_Phase_2.to_csv("Processed_Dialyse_Phase_2.csv", index=False)
Processed_Dialyse_Phase_3.to_csv("Processed_Dialyse_Phase_3.csv", index=False)
Processed_Dialyse_Phase_4.to_csv("Processed_Dialyse_Phase_4.csv", index=False)
Processed_Dialyse_Phase_5.to_csv("Processed_Dialyse_Phase_5.csv", index=False)

Labsvar

In [ ]:
def Lab_svarProcessing1(filtered_Lab_svar_1,filtered_Lab_svar_2):
    # print(filtered_Lab_svar_1)
    ja = filtered_Lab_svar_1.drop(columns = 'Timestamp')
    # print("ja")
    # print(ja)
    grouped = ja.groupby('IDno')
    # print("Grouped:")
    # print(grouped)
    agg_dict = {col: ['mean', 'var'] for col in ja.columns if col != 'IDno'}
    # print("agg_dict:")
    # print(agg_dict)
    result = grouped.agg(agg_dict)
    # print("Result:")
    # print(result)
    result.columns = ['_'.join(col).strip() for col in result.columns.values]
    for col in result.columns:
        if 'var' in col:
            mean_col = col.replace('var', 'mean')
            result[col] = result.apply(lambda row: 0 if pd.notna(row[mean_col]) and pd.isna(row[col]) else row[col], axis=1)
    Processed_Lab_svar_1 = result
    # print("Processed lab svar 1:")
    # print(Processed_Lab_svar_1)

    ja = filtered_Lab_svar_2.drop(columns = 'Timestamp')
    grouped = ja.groupby('IDno')
    agg_dict = {col: ['mean', 'var'] for col in ja.columns if col != 'IDno'}
    result = grouped.agg(agg_dict)
    result.columns = ['_'.join(col).strip() for col in result.columns.values]
    for col in result.columns:
        if 'var' in col:
            mean_col = col.replace('var', 'mean')
            result[col] = result.apply(lambda row: 0 if pd.notna(row[mean_col]) and pd.isna(row[col]) else row[col], axis=1)
    for col in result.columns:
        if 'mean' in col:
            var_col = col.replace('mean', 'var')
            valid_col = col.replace('mean', 'valid')
            result[valid_col] = result.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)
    Processed_Lab_svar_2 = result

    kol_navn = []
    for i in range(len(Processed_Lab_svar_1.columns)):
        if list(Processed_Lab_svar_1.columns)[i] not in kol_navn:
            kol_navn.append(list(Processed_Lab_svar_1.columns)[i])
    for i in range(len(Lab_svar_2.columns)):
        if list(Processed_Lab_svar_2.columns)[i] not in kol_navn:
            kol_navn.append(list(Processed_Lab_svar_2.columns)[i])
    kol_navn = kol_navn[2:]
    # print("Kol navn:")
    # print(kol_navn)
    ny_list = {}
    for i in range(len(kol_navn)):
        tal = 0
        if kol_navn[i] in Processed_Lab_svar_1:
            tal = tal + (len(Processed_Lab_svar_1[~Processed_Lab_svar_1[kol_navn[i]].isna()]))
        if kol_navn[i] in Processed_Lab_svar_2:
            tal = tal + (len(Processed_Lab_svar_2[~Processed_Lab_svar_2[kol_navn[i]].isna()]))
        ny_list[kol_navn[i]] = tal
    
    # print("Ny list:")
    # print(len(ny_list))
    # print(ny_list)
    return Processed_Lab_svar_1, Processed_Lab_svar_2, kol_navn,ny_list


def Lab_svarProcessing2(Processed_Lab_svar_1, Processed_Lab_svar_2, kol_navn,ny_list):
    antal = 0
    hvilke = []
    for i in range(len(ny_list)):
        if ny_list[kol_navn[i]] >= 500:
            antal += 1
            hvilke.append(kol_navn[i])
    antal = antal / 2
    # print('Hvilke:')
    # print(hvilke)

    Processed_Lab_svar_1 = Processed_Lab_svar_1[hvilke]
    Processed_Lab_svar_2 = Processed_Lab_svar_2[hvilke]
    # print("Processed_lab_svar_1[hvilke]:")
    # print(Processed_Lab_svar_1)

    return Processed_Lab_svar_1, Processed_Lab_svar_2, hvilke, antal

def Lab_svarProcessing3(Processed_Lab_svar_1, Processed_Lab_svar_2):
    for col in Processed_Lab_svar_1.columns:
        if 'mean' in col:
            var_col = col.replace('mean', 'var')
            valid_col = col.replace('mean', 'valid')
            Processed_Lab_svar_1[valid_col] = Processed_Lab_svar_1.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)
    for col in Processed_Lab_svar_2.columns:
        if 'mean' in col:
            var_col = col.replace('mean', 'var')
            valid_col = col.replace('mean', 'valid')
            Processed_Lab_svar_2[valid_col] = Processed_Lab_svar_2.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)

    Processed_Lab_svar_1 = Processed_Lab_svar_1.fillna(0)
    Processed_Lab_svar_2 = Processed_Lab_svar_2.fillna(0)
    Processed_Lab_svar_1.reset_index(inplace = True)
    Processed_Lab_svar_2.reset_index(inplace = True)
    Processed_Lab_svar = pd.concat([Processed_Lab_svar_1,Processed_Lab_svar_2], ignore_index = True)

    Processed_Lab_svar = Processed_Lab_svar.applymap(lambda x: 1 if x == True else (0 if x == False else x))
    # print("Processed_Lab_svar:")
    # print(Processed_Lab_svar)

    # Step 2: Add missing IDno from Populationen with all other columns filled with 0
    missing_ids = IDS_dfx.loc[~IDS_dfx['IDno'].isin(Processed_Lab_svar['IDno']), 'IDno']
    # print("Missing ids:")
    # print(missing_ids)
    missing_data = pd.DataFrame({'IDno': missing_ids})
    # print("Missing data:")
    # print(missing_data)
    columns_to_add = [col for col in Processed_Lab_svar.columns if col != 'IDno']
    missing_data[columns_to_add] = 0

    Processed_Lab_svar = pd.concat([Processed_Lab_svar, missing_data], ignore_index=True)
    Processed_Lab_svar = Processed_Lab_svar.groupby('IDno').max().reset_index()
    pre_liste.append(Processed_Lab_svar)

    return Processed_Lab_svar

In [ ]:
Processed_Lab_svar_1_Phase_1, Processed_Lab_svar_2_Phase_1, kol_navn_Phase_1,ny_list_Phase_1 = Lab_svarProcessing1(Lab_svar_1_Phase_1,Lab_svar_2_Phase_1)
Processed_Lab_svar_1_Phase_2, Processed_Lab_svar_2_Phase_2, kol_navn_Phase_2,ny_list_Phase_2 = Lab_svarProcessing1(Lab_svar_1_Phase_2,Lab_svar_2_Phase_2)
Processed_Lab_svar_1_Phase_3, Processed_Lab_svar_2_Phase_3, kol_navn_Phase_3,ny_list_Phase_3 = Lab_svarProcessing1(Lab_svar_1_Phase_3,Lab_svar_2_Phase_3)
Processed_Lab_svar_1_Phase_4, Processed_Lab_svar_2_Phase_4, kol_navn_Phase_4,ny_list_Phase_4 = Lab_svarProcessing1(Lab_svar_1_Phase_4,Lab_svar_2_Phase_4)
Processed_Lab_svar_1_Phase_5, Processed_Lab_svar_2_Phase_5, kol_navn_Phase_5,ny_list_Phase_5 = Lab_svarProcessing1(Lab_svar_1_Phase_5,Lab_svar_2_Phase_5)

In [ ]:
Processed_Lab_svar_1_Phase_1, Processed_Lab_svar_2_Phase_1, hvilke_Phase_1, antal_Phase_1 = Lab_svarProcessing2(Processed_Lab_svar_1_Phase_1, Processed_Lab_svar_2_Phase_1, kol_navn_Phase_1,ny_list_Phase_1)
Processed_Lab_svar_1_Phase_2, Processed_Lab_svar_2_Phase_2, hvilke_Phase_2, antal_Phase_2 = Lab_svarProcessing2(Processed_Lab_svar_1_Phase_2, Processed_Lab_svar_2_Phase_2, kol_navn_Phase_2,ny_list_Phase_2)
Processed_Lab_svar_1_Phase_3, Processed_Lab_svar_2_Phase_3, hvilke_Phase_3, antal_Phase_3 = Lab_svarProcessing2(Processed_Lab_svar_1_Phase_3, Processed_Lab_svar_2_Phase_3, kol_navn_Phase_3,ny_list_Phase_3)
Processed_Lab_svar_1_Phase_4, Processed_Lab_svar_2_Phase_4, hvilke_Phase_4, antal_Phase_4 = Lab_svarProcessing2(Processed_Lab_svar_1_Phase_4, Processed_Lab_svar_2_Phase_4, kol_navn_Phase_4,ny_list_Phase_4)
Processed_Lab_svar_1_Phase_5, Processed_Lab_svar_2_Phase_5, hvilke_Phase_5, antal_Phase_5 = Lab_svarProcessing2(Processed_Lab_svar_1_Phase_5, Processed_Lab_svar_2_Phase_5, kol_navn_Phase_5,ny_list_Phase_5)

In [ ]:
Processed_Lab_svar_Phase_1 = Lab_svarProcessing3(Processed_Lab_svar_1_Phase_1, Processed_Lab_svar_2_Phase_1)
Processed_Lab_svar_Phase_2 = Lab_svarProcessing3(Processed_Lab_svar_1_Phase_2, Processed_Lab_svar_2_Phase_2)
Processed_Lab_svar_Phase_3 = Lab_svarProcessing3(Processed_Lab_svar_1_Phase_3, Processed_Lab_svar_2_Phase_3)
Processed_Lab_svar_Phase_4 = Lab_svarProcessing3(Processed_Lab_svar_1_Phase_4, Processed_Lab_svar_2_Phase_4)
Processed_Lab_svar_Phase_5 = Lab_svarProcessing3(Processed_Lab_svar_1_Phase_5, Processed_Lab_svar_2_Phase_5)

In [ ]:
Processed_Lab_svar_Phase_1.to_csv("Phases/Processed_Lab_svar_Phase_1.csv", index=False)
Processed_Lab_svar_Phase_2.to_csv("Phases/Processed_Lab_svar_Phase_2.csv", index=False)
Processed_Lab_svar_Phase_3.to_csv("Phases/Processed_Lab_svar_Phase_3.csv", index=False)
Processed_Lab_svar_Phase_4.to_csv("Phases/Processed_Lab_svar_Phase_4.csv", index=False)
Processed_Lab_svar_Phase_5.to_csv("Phases/Processed_Lab_svar_Phase_5.csv", index=False)

Skal ændres: AHM, Anæstesidata, Blødning, Total indgift, 
Skal indkluderes: Dialyse, Labsvar

AHM Processed

In [ ]:
def AHMProcessing(AHM):
    # Kolonner, der har mindre end 1000 målinger i hele AHM
    columns_to_drop = ['Timestamp', 'ABP inv BT', 'AO Puls (fra A-kanyle)', 'AO inv BT', 'AO mean inv BT', 'ART Puls (fra A-kanyle)', 'ART inv BT', 'Art', 'Blodtryk', 'CCI', 'CCO', 'CI', 'CO', 'CPP', 'CVP (sys/dia)',
    'EVLW', 'EVLWi', 'GEDV', 'GEDVi', 'Invasivt BT - ABP (sys/dia)', 'Invasivt BT - AO (middel)', 'Invasivt BT - AO (sys/dia)', 'Invasivt BT - ART (mean)', 'Invasivt BT - ART (sys/dia)', 'Invasivt BT - UAP (middel)',
    'Invasivt BT - UAP (sys/dia)', 'LAP', 'LAP (middel)', 'LAP (sys/dia)', 'NIBP', 'PAP', 'PAP (sys/dia)', 'PCWP', 'PPV', 'PVR', 'PVRI', 'Puls (fra A-kanyle - ART)', 'Puls 2 (fra A-kanyle - UAP)', 'Puls amplitude',
    'RAP (middel)', 'RAP (sys/dia)', 'SATvO2', 'SVR', 'SVRI', 'SVi', 'Saturation, fod (postduktal)', 'Saturation, højre arm (præduktal)', 'Slagvolumen (SV)', 'Slagvolumenvariation (SVV)', 'Slagvolumenvariation LIDCO',
    'Sondedybde (cm)', 'SpO2 L', 'SpO2 R', 'SvO2', 'Temperatur 1', 'Temperatur 2', 'Temperatur 3', 'Temperatur 4', 'Temperatur, højre', 'Temperatur, venstre','RF - indstillet']
    # RF - indstillet fjernet fordi den er det samme for alle
    Beginning_AHM = AHM.drop(columns=columns_to_drop)
    
    remove_more_columns = []
    non_null_counts = Beginning_AHM.groupby('IDno').count()

    # Filter columns with all null values for each group
    columns_with_all_nulls = non_null_counts.columns[(non_null_counts == 0).all()].tolist()
    non_null_counts_no_zeros = non_null_counts.replace(0, np.nan)
    count_AHM_pr_ID = non_null_counts_no_zeros.count()

    for column, amounts in count_AHM_pr_ID.items():
        if amounts/len(Beginning_AHM['IDno'].unique())*100 < 80: # Måske en anden værdi
            remove_more_columns.append(column)
    
    if len(remove_more_columns)>0:
        Beginning_AHM = Beginning_AHM.drop(columns=remove_more_columns)

    dummies = pd.get_dummies(Beginning_AHM['Skema navn']).astype(int)

    skema_titler = list(dummies.columns)

    Beginning_AHM = pd.concat([Beginning_AHM, dummies], axis = 1)

    Beginning_AHM = Beginning_AHM.drop(columns = 'Skema navn')

    original_columns = [col for col in Beginning_AHM.columns if col not in ['IDno'] + skema_titler]

    grouped = Beginning_AHM.groupby('IDno')[original_columns].agg(['mean', 'var'])

    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

    Processed_AHM = grouped.reset_index()    

    dummies_grouped = Beginning_AHM.groupby('IDno')[list(dummies.columns)].max().reset_index()

    Processed_AHM = pd.merge(Processed_AHM,dummies_grouped, on='IDno')

    missing_idnos = IDS_dfx[~IDS_dfx['IDno'].isin(Processed_AHM['IDno'])]['IDno']

    # Create a dataframe with these missing IDnos and 0 in every other input
    missing_data = pd.DataFrame(missing_idnos, columns=['IDno'])
    for col in Processed_AHM.columns:
        if col != 'IDno':
            missing_data[col] = 0

    # Append this missing data to Processed_AHM
    Processed_AHM = pd.concat([Processed_AHM, missing_data], ignore_index=True)

    # Replace NaN values with 0
    Processed_AHM.fillna(0, inplace=True)
    Processed_AHM = Processed_AHM.groupby('IDno').max().reset_index()
    
    return Processed_AHM        
        
    # Start med at tjekke om alle værdierne i en kolonne er NaN, if so kan den nok godt droppes
    # Ved ikke, hvordan skema navn skal håndteres, skriv en mail til Theis og Lars
    # Der er nogle kolonner, der kun har meget få målinger, kan evt. være at de skal undlades. Hver fase har forskellige mængder rækker, så måske en procentdel? Findes med statistik? Fase 1 har meget få målinger, de andre er bedre
    # På dette tidspunkt er vi nede på maks 27 kolonner, som der nok skal bestemmes mean og varians af. Måske ok mængde? Tror, det er færre end Lab svar
    
    

In [ ]:
Processed_AHM_Phase_1 = AHMProcessing(AHM_Phase_1)
Processed_AHM_Phase_2 = AHMProcessing(AHM_Phase_2)
Processed_AHM_Phase_3 = AHMProcessing(AHM_Phase_3)
Processed_AHM_Phase_4 = AHMProcessing(AHM_Phase_4)
Processed_AHM_Phase_5 = AHMProcessing(AHM_Phase_5)

In [ ]:
Processed_AHM_Phase_1.to_csv("Processed_AHM_Phase_1.csv", index=False)
Processed_AHM_Phase_2.to_csv("Processed_AHM_Phase_2.csv", index=False)
Processed_AHM_Phase_3.to_csv("Processed_AHM_Phase_3.csv", index=False)
Processed_AHM_Phase_4.to_csv("Processed_AHM_Phase_4.csv", index=False)
Processed_AHM_Phase_5.to_csv("Processed_AHM_Phase_5.csv", index=False)

In [ ]:
len(non_null_counts[non_null_counts > 1000].index.tolist())

In [ ]:
non_null_counts = AHM.groupby('IDno').count()

# Filter columns with all null values for each group
columns_with_all_nulls = non_null_counts.columns[(non_null_counts == 0).all()].tolist()
non_null_counts_no_zeros = non_null_counts.replace(0, np.nan)
count_AHM_pr_ID = non_null_counts_no_zeros.count()

# Set pandas display options to show all columns
# pd.set_option('display.max_columns', None)  # To display all columns
# pd.set_option('display.max_rows', None)  # To display all rows if needed

# Print the non-zero counts
print("Count of non-zero values in each column:")
print(count_AHM_pr_ID)

In [ ]:
(count_AHM_pr_ID[count_AHM_pr_ID < 1000].index.tolist())

In [ ]:
# Kolonner, der har mindre end 1000 målinger i hele AHM
columns_to_drop = ['Timestamp', 'ABP inv BT', 'AO Puls (fra A-kanyle)', 'AO inv BT', 'AO mean inv BT', 'ART Puls (fra A-kanyle)', 'ART inv BT', 'Art', 'Blodtryk', 'CCI', 'CCO', 'CI', 'CO', 'CPP', 'CVP (sys/dia)',
    'EVLW', 'EVLWi', 'GEDV', 'GEDVi', 'Invasivt BT - ABP (sys/dia)', 'Invasivt BT - AO (middel)', 'Invasivt BT - AO (sys/dia)', 'Invasivt BT - ART (mean)', 'Invasivt BT - ART (sys/dia)', 'Invasivt BT - UAP (middel)',
    'Invasivt BT - UAP (sys/dia)', 'LAP', 'LAP (middel)', 'LAP (sys/dia)', 'NIBP', 'PAP', 'PAP (sys/dia)', 'PCWP', 'PPV', 'PVR', 'PVRI', 'Puls (fra A-kanyle - ART)', 'Puls 2 (fra A-kanyle - UAP)', 'Puls amplitude',
    'RAP (middel)', 'RAP (sys/dia)', 'SATvO2', 'SVR', 'SVRI', 'SVi', 'Saturation, fod (postduktal)', 'Saturation, højre arm (præduktal)', 'Slagvolumen (SV)', 'Slagvolumenvariation (SVV)', 'Slagvolumenvariation LIDCO',
    'Sondedybde (cm)', 'SpO2 L', 'SpO2 R', 'SvO2', 'Temperatur 1', 'Temperatur 2', 'Temperatur 3', 'Temperatur 4', 'Temperatur, højre', 'Temperatur, venstre']
len(columns_to_drop)

In [ ]:
AHM_cleaned = AHM.drop(columns=columns_to_drop)

In [ ]:
AHM_cleaned_Phase_1 = AHM_Phase_1.drop(columns=columns_to_drop)

In [ ]:
AHM_cleaned_Phase_2 = AHM_Phase_2.drop(columns=columns_to_drop)
AHM_cleaned_Phase_3 = AHM_Phase_3.drop(columns=columns_to_drop)
AHM_cleaned_Phase_4 = AHM_Phase_4.drop(columns=columns_to_drop)
AHM_cleaned_Phase_5 = AHM_Phase_5.drop(columns=columns_to_drop)

In [ ]:
non_null_counts = AHM_cleaned_Phase_5.groupby('IDno').count()

# Filter columns with all null values for each group
columns_with_all_nulls = non_null_counts.columns[(non_null_counts == 0).all()].tolist()
non_null_counts_no_zeros = non_null_counts.replace(0, np.nan)
count_AHM_pr_ID = non_null_counts_no_zeros.count()

# Set pandas display options to show all columns
# pd.set_option('display.max_columns', None)  # To display all columns
# pd.set_option('display.max_rows', None)  # To display all rows if needed

# Print the non-zero counts
# print("Count of non-zero values in each column:")
# print(count_AHM_pr_ID)

In [ ]:
remove_more_columns = []

for column, amounts in (count_AHM_pr_ID).items():
    print(column)
    print(amounts/len(AHM_cleaned_Phase_5['IDno'].unique())*100) # Måske en anden værdi
        

Anæstesidata Processed

In [ ]:
Anæstesidata

In [ ]:
def AnæstesidataProcessing(Anæstesidata):
    Beginning_Anæstesidata = Anæstesidata
    Beginning_Anæstesidata[['ABP inv BT Systolisk','ABP inv BT Diastolisk']] = Beginning_Anæstesidata['ABP inv BT'].str.split('/', expand = True).astype(float)
    
    remove_more_columns = ['Timestamp','ABP inv BT']
    non_null_counts = Beginning_Anæstesidata.groupby('IDno').count()

    # Filter columns with all null values for each group
    columns_with_all_nulls = non_null_counts.columns[(non_null_counts == 0).all()].tolist()
    non_null_counts_no_zeros = non_null_counts.replace(0, np.nan)
    count_AHM_pr_ID = non_null_counts_no_zeros.count()

    for column, amounts in count_AHM_pr_ID.items():
        if amounts/len(Beginning_Anæstesidata['IDno'].unique())*100 < 80: # Måske en anden værdi
            remove_more_columns.append(column)
    
    if len(remove_more_columns)>0:
        Beginning_Anæstesidata = Beginning_Anæstesidata.drop(columns=remove_more_columns)

    # List of columns to calculate mean and variance
    columns_to_aggregate = [col for col in Beginning_Anæstesidata.columns if col not in ['IDno']]

    # Calculate mean and variance grouped by 'IDno'
    grouped = Beginning_Anæstesidata.groupby('IDno')[columns_to_aggregate].agg(['mean', 'var'])
    
    # Flatten the hierarchical column index
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

    # Reset index to bring 'IDno' back as a column
    Processed_Anæstesidata = grouped.reset_index()
    
    missing_idnos = IDS_dfx[~IDS_dfx['IDno'].isin(Processed_Anæstesidata['IDno'])]['IDno']

    # Create a dataframe with these missing IDnos and 0 in every other input
    missing_data = pd.DataFrame(missing_idnos, columns=['IDno'])
    for col in Processed_Anæstesidata.columns:
        if col != 'IDno':
            missing_data[col] = 0

    # Append this missing data to Processed_Anæstesidata
    Processed_Anæstesidata = pd.concat([Processed_Anæstesidata, missing_data], ignore_index=True)

    # Replace NaN values with 0
    Processed_Anæstesidata.fillna(0, inplace=True)
    Processed_Anæstesidata = Processed_Anæstesidata.groupby('IDno').max().reset_index()
    
    return Processed_Anæstesidata

In [ ]:
Processed_Anæstesidata_Phase_1 = AnæstesidataProcessing(Anæstesidata_Phase_1)
Processed_Anæstesidata_Phase_2 = AnæstesidataProcessing(Anæstesidata_Phase_2)
Processed_Anæstesidata_Phase_3 = AnæstesidataProcessing(Anæstesidata_Phase_3)
Processed_Anæstesidata_Phase_4 = AnæstesidataProcessing(Anæstesidata_Phase_4)
Processed_Anæstesidata_Phase_5 = AnæstesidataProcessing(Anæstesidata_Phase_5)

In [ ]:
Processed_Anæstesidata_Phase_1.to_csv("Phases/Processed_Anæstesidata_Phase_1.csv", index=False)
Processed_Anæstesidata_Phase_2.to_csv("Phases/Processed_Anæstesidata_Phase_2.csv", index=False)
Processed_Anæstesidata_Phase_3.to_csv("Phases/Processed_Anæstesidata_Phase_3.csv", index=False)
Processed_Anæstesidata_Phase_4.to_csv("Phases/Processed_Anæstesidata_Phase_4.csv", index=False)
Processed_Anæstesidata_Phase_5.to_csv("Phases/Processed_Anæstesidata_Phase_5.csv", index=False)

Blødning Processed

In [ ]:
def BlødningProcessing(Blødning):
    Beginning_Blødning = Blødning
    
    remove_columns = ['Timestamp']

    Beginning_Blødning = Beginning_Blødning.drop(columns = remove_columns)

    # List of columns to calculate mean and variance
    columns_to_aggregate = [col for col in Beginning_Blødning.columns if col not in ['IDno']]

    # Calculate mean and variance grouped by 'IDno'
    grouped = Beginning_Blødning.groupby('IDno')[columns_to_aggregate].agg(['mean', 'var'])
    
    # Flatten the hierarchical column index
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

    # Reset index to bring 'IDno' back as a column
    Processed_Blødning = grouped.reset_index()
    
    missing_idnos = IDS_dfx[~IDS_dfx['IDno'].isin(Processed_Blødning['IDno'])]['IDno']

    # Create a dataframe with these missing IDnos and 0 in every other input
    missing_data = pd.DataFrame(missing_idnos, columns=['IDno'])
    for col in Processed_Blødning.columns:
        if col != 'IDno':
            missing_data[col] = 0

    # Append this missing data to Processed_Blødning
    Processed_Blødning = pd.concat([Processed_Blødning, missing_data], ignore_index=True)

    # Replace NaN values with 0
    Processed_Blødning.fillna(0, inplace=True)
    Processed_Blødning = Processed_Blødning.groupby('IDno').max().reset_index()
    
    return Processed_Blødning

In [ ]:
Processed_Blødning_Phase_1 = BlødningProcessing(Blødning_Phase_1)
Processed_Blødning_Phase_2 = BlødningProcessing(Blødning_Phase_2)
Processed_Blødning_Phase_3 = BlødningProcessing(Blødning_Phase_3)
Processed_Blødning_Phase_4 = BlødningProcessing(Blødning_Phase_4)
Processed_Blødning_Phase_5 = BlødningProcessing(Blødning_Phase_5)

In [ ]:
Processed_Blødning_Phase_1.to_csv("Phases/Processed_Blødning_Phase_1.csv", index=False)
Processed_Blødning_Phase_2.to_csv("Phases/Processed_Blødning_Phase_2.csv", index=False)
Processed_Blødning_Phase_3.to_csv("Phases/Processed_Blødning_Phase_3.csv", index=False)
Processed_Blødning_Phase_4.to_csv("Phases/Processed_Blødning_Phase_4.csv", index=False)
Processed_Blødning_Phase_5.to_csv("Phases/Processed_Blødning_Phase_5.csv", index=False)

In [ ]:
Blødning.columns

In [ ]:
Blødning_Phase_1['Blødning ml'].dtype

Total indgift Processed

In [ ]:
Total_indgift.columns

In [ ]:
def Total_indgiftProcessing(Total_indgift):
    Beginning_Total_indgift = Total_indgift

    Beginning_Total_indgift = Beginning_Total_indgift.drop(columns = ['Kommentar','Timestamp'])

    dummies1 = pd.get_dummies(Beginning_Total_indgift['Ordineret lægemiddel navn']).astype(int)

    dummies2 = pd.get_dummies(Beginning_Total_indgift['Række navn']).astype(int)

    skema_titler1 = list(dummies1.columns)

    skema_titler2 = list(dummies2.columns)

    Beginning_Total_indgift = pd.concat([Beginning_Total_indgift, dummies1, dummies2], axis = 1)

    Beginning_Total_indgift = Beginning_Total_indgift.drop(columns = ['Ordineret lægemiddel navn', 'Række navn'])

    original_columns = [col for col in Beginning_Total_indgift.columns if col not in ['IDno'] + skema_titler1 + skema_titler2]

    grouped = Beginning_Total_indgift.groupby('IDno')[original_columns].agg(['mean', 'var'])

    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

    Processed_Total_indgift = grouped.reset_index()    

    dummies_grouped1 = Beginning_Total_indgift.groupby('IDno')[list(dummies1.columns)].max().reset_index()

    dummies_grouped2 = Beginning_Total_indgift.groupby('IDno')[list(dummies2.columns)].max().reset_index()

    Processed_Total_indgift = pd.merge(Processed_Total_indgift,dummies_grouped1, on='IDno')

    Processed_Total_indgift = pd.merge(Processed_Total_indgift,dummies_grouped2, on='IDno')

    missing_idnos = IDS_dfx[~IDS_dfx['IDno'].isin(Processed_Total_indgift['IDno'])]['IDno']

    # Create a dataframe with these missing IDnos and 0 in every other input
    missing_data = pd.DataFrame(missing_idnos, columns=['IDno'])
    for col in Processed_Total_indgift.columns:
        if col != 'IDno':
            missing_data[col] = 0

    # Append this missing data to Processed_Total_indgift
    Processed_Total_indgift = pd.concat([Processed_Total_indgift, missing_data], ignore_index=True)

    # Replace NaN values with 0
    Processed_Total_indgift.fillna(0, inplace=True)
    Processed_Total_indgift = Processed_Total_indgift.groupby('IDno').max().reset_index()
    
    return Processed_Total_indgift        


In [ ]:
Processed_Total_indgift_Phase_1 = Total_indgiftProcessing(Total_indgift_Phase_1)
Processed_Total_indgift_Phase_2 = Total_indgiftProcessing(Total_indgift_Phase_2)
Processed_Total_indgift_Phase_3 = Total_indgiftProcessing(Total_indgift_Phase_3)
Processed_Total_indgift_Phase_4 = Total_indgiftProcessing(Total_indgift_Phase_4)
Processed_Total_indgift_Phase_5 = Total_indgiftProcessing(Total_indgift_Phase_5)

In [ ]:
Processed_Total_indgift_Phase_1.to_csv("Phases/Processed_Total_indgift_Phase_1.csv", index=False)
Processed_Total_indgift_Phase_2.to_csv("Phases/Processed_Total_indgift_Phase_2.csv", index=False)
Processed_Total_indgift_Phase_3.to_csv("Phases/Processed_Total_indgift_Phase_3.csv", index=False)
Processed_Total_indgift_Phase_4.to_csv("Phases/Processed_Total_indgift_Phase_4.csv", index=False)
Processed_Total_indgift_Phase_5.to_csv("Phases/Processed_Total_indgift_Phase_5.csv", index=False)

In [ ]:
Processed_Total_indgift_Phase_1

In [ ]:
len(Total_indgift_Phase_1['Ordineret lægemiddel navn'].unique()),len(Total_indgift_Phase_1['Række navn'].unique())

In [ ]:
print(len(Total_indgift_Phase_2['Ordineret lægemiddel navn'].unique()),len(Total_indgift_Phase_2['Række navn'].unique()))
print(len(Total_indgift_Phase_3['Ordineret lægemiddel navn'].unique()),len(Total_indgift_Phase_3['Række navn'].unique()))
print(len(Total_indgift_Phase_4['Ordineret lægemiddel navn'].unique()),len(Total_indgift_Phase_4['Række navn'].unique()))
print(len(Total_indgift_Phase_5['Ordineret lægemiddel navn'].unique()),len(Total_indgift_Phase_5['Række navn'].unique()))

In [ ]:
test = Total_indgift.copy()
test = test.fillna('')
test['Ordineret og Rækken navn'] = test['Række navn']+test['Ordineret lægemiddel navn']
len(test['Ordineret og Rækken navn'].unique()),len(test['Ordineret lægemiddel navn'].unique()),len(test['Række navn'].unique())

In [ ]:
remove_more_columns = []
non_null_counts = Total_indgift_Phase_1.groupby('IDno').count()

# Filter columns with all null values for each group
columns_with_all_nulls = non_null_counts.columns[(non_null_counts == 0).all()].tolist()
non_null_counts_no_zeros = non_null_counts.replace(0, np.nan)
count_AHM_pr_ID = non_null_counts_no_zeros.count()

for column, amounts in count_AHM_pr_ID.items():
    if amounts/len(Total_indgift_Phase_1['IDno'].unique())*100 < 80: # Måske en anden værdi
        remove_more_columns.append(column)
        print(amounts)
        

In [ ]:
test

In [ ]:
Total_indgift_Phase_1[Total_indgift_Phase_1['Ordineret lægemiddel navn'].isna()]

In [ ]:
count = 0
liste = []
for item in sorted(list(Total_indgift_Phase_1[Total_indgift_Phase_1['Ordineret lægemiddel navn'].isna()]['Værdi'])):
    if item < 1:
        count +=1
print(count)

In [ ]:
Total_indgift_Phase_1

X matricer

In [65]:
from functools import reduce
dataframes_to_merge = [Processed_AHM_Phase_1, Processed_Anæstesidata_Phase_1, Processed_Lab_svar_Phase_1, Processed_Total_indgift_Phase_1]

x_matrix_phase_1 = reduce(lambda left, right: pd.merge(left, right, on='IDno', how='outer'), dataframes_to_merge)
x_matrix_phase_1 = x_matrix_phase_1.replace('Kvinde',0)
x_matrix_phase_1 = x_matrix_phase_1.replace('Mand',1)

In [66]:
dataframes_to_merge = [Processed_AHM_Phase_2, Processed_Anæstesidata_Phase_2, Processed_Lab_svar_Phase_2, Processed_Total_indgift_Phase_2]

x_matrix_phase_2 = reduce(lambda left, right: pd.merge(left, right, on='IDno', how='outer'), dataframes_to_merge)
x_matrix_phase_2 = x_matrix_phase_2.replace('Kvinde',0)
x_matrix_phase_2 = x_matrix_phase_2.replace('Mand',1)

In [67]:
dataframes_to_merge = [Processed_AHM_Phase_3, Processed_Anæstesidata_Phase_3, Processed_Lab_svar_Phase_3, Processed_Total_indgift_Phase_3]

x_matrix_phase_3 = reduce(lambda left, right: pd.merge(left, right, on='IDno', how='outer'), dataframes_to_merge)
x_matrix_phase_3 = x_matrix_phase_3.replace('Kvinde',0)
x_matrix_phase_3 = x_matrix_phase_3.replace('Mand',1)

In [68]:
dataframes_to_merge = [Processed_AHM_Phase_4, Processed_Anæstesidata_Phase_4, Processed_Lab_svar_Phase_4, Processed_Total_indgift_Phase_4]

x_matrix_phase_4 = reduce(lambda left, right: pd.merge(left, right, on='IDno', how='outer'), dataframes_to_merge)
x_matrix_phase_4 = x_matrix_phase_4.replace('Kvinde',0)
x_matrix_phase_4 = x_matrix_phase_4.replace('Mand',1)

In [69]:
dataframes_to_merge = [Processed_AHM_Phase_5, Processed_Anæstesidata_Phase_5, Processed_Lab_svar_Phase_5, Processed_Total_indgift_Phase_5]

x_matrix_phase_5 = reduce(lambda left, right: pd.merge(left, right, on='IDno', how='outer'), dataframes_to_merge)
x_matrix_phase_5 = x_matrix_phase_5.replace('Kvinde',0)
x_matrix_phase_5 = x_matrix_phase_5.replace('Mand',1)

In [71]:
x_matrix_phase_1.to_csv('x_matrix_phase_1_4738.csv',compression='gzip',index=False)
x_matrix_phase_2.to_csv('x_matrix_phase_2_4738.csv',compression='gzip',index=False)
x_matrix_phase_3.to_csv('x_matrix_phase_3_4738.csv',compression='gzip',index=False)
x_matrix_phase_4.to_csv('x_matrix_phase_4_4738.csv',compression='gzip',index=False)
x_matrix_phase_5.to_csv('x_matrix_phase_5_4738.csv',compression='gzip',index=False)